# Install the bqplot notebook extension

In [ ]:
from __future__ import print_function
from bqplot.install import install
install(user=True, symlink=True)

# A simple plot with the pyplot API

In [ ]:
import numpy as np
from bqplot import pyplot as plt

In [ ]:
plt.figure(1)
n = 100
plt.plot(np.linspace(0.0, 10.0, n), np.cumsum(np.random.randn(n)), axes_options={'y': {'grid_lines': 'dashed'}})
plt.show()

# A more verbose and atomic API for creating charts

In [ ]:
size = 100
x_data = np.random.randn(size)
y_data = np.cumsum(np.random.randn(size)  * 100.0)
color_data = np.cumsum(np.random.randn(size))
rotation_data = np.sin(x_data * 0.1)

In [ ]:
from bqplot import *

x_sc = LinearScale()
y_sc = LinearScale()
c_sc = ColorScale()
r_sc = LinearScale()
s_sc = LinearScale()

ax_x = Axis(label='X Axis', scale=x_sc, grid_lines='solid')
ax_y = Axis(label='Y Axis', scale=y_sc, orientation='vertical', grid_lines='solid')
ax_c = ColorAxis(label='Color Scale', scale=c_sc, orientation='horizontal')

scatter = Scatter(x=x_data,
                  y=y_data,
                  color=color_data,
                  rotation=rotation_data,
                  marker='arrow',
                  animate_dur=1000,
                  default_size=100,
                  scales={'x': x_sc, 'y': y_sc, 'color': c_sc, 'rotation': r_sc, 'size': s_sc})

fig = Figure(axes=[ax_x, ax_y, ax_c], marks=[scatter])
fig

# Each item of a figure is an IPython widget and reflects changes of its attributes

In [ ]:
# Dynamically changing the color data
scatter.color = x_data

In [ ]:
# Dynamically changing the marker rotation
scatter.rotation = np.cos(x_data)

In [ ]:
# Dynamically changing the domain of the x scale
x_sc.min = -5

In [ ]:
# Setting the minimum of the x scale back to None
x_sc.min = None

In [ ]:
# Changing the color scheme (Color Brewer schemes)
c_sc.scheme = 'Purples'

In [ ]:
# Displaying another view of the same figure
fig

# Wiring bqplot widgets with other IPython widgets

In [ ]:
from ipywidgets import FloatSlider

# Wiring the rotation parameter with a slider
def handle_slider(name, old, new):
    scatter.rotation = np.cos(10 * new * x_data)
    scatter.size = np.sin(10 * new * x_data)

handle_slider('value', 0.5, 0.5)
slider = FloatSlider(value=0.5, min=0, max=1.0, step=0.01, description='Adjust rotation frequency')
slider.on_trait_change(handle_slider, name='value')

slider

# More advanced visualization: Historical GDP per Capita of different countries

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('country_codes.csv', index_col=[0])
country_codes = data.index.values
country_names = data['Name']

gdp_data = pd.read_csv('gdp_per_capita.csv', index_col=[0], parse_dates=True)
gdp_data.fillna(method='backfill', inplace=True)
gdp_data.fillna(method='ffill', inplace=True)

data['GDP'] = gdp_data.ix[-1]

continents = data['Continent'].values

In [ ]:
# Creating the figure to be displayed as the tooltip
sc_x = DateScale()
sc_y = LinearScale()
sc_c = ColorScale(scheme='Greens')

ax_x = Axis(scale=sc_x, grid_lines='dashed', label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', grid_lines='dashed',
            label='GDP', label_location='end', label_offset='-1em')
ax_c = ColorAxis(scale=sc_c)

line = Lines(x= gdp_data.index.values, scales={'x': sc_x, 'y': sc_y}, colors=['orange'])
fig_tooltip = Figure(marks=[line], axes=[ax_x, ax_y], min_width=600, min_height=400)

In [ ]:
from bqplot.market_map import MarketMap

market_map = MarketMap(names=country_codes, groups=continents,
                       cols=25, row_groups=3,
                       color=data['GDP'], scales={'color': sc_c}, axes=[ax_c],
                       ref_data=data, tooltip_widget=fig_tooltip)

# Update the tooltip
hovered_symbol = ''
def hover_handler(self, content):
    global hovered_symbol
    symbol = content.get('ref_data', {}).get('Country Code', '')
    if(symbol != hovered_symbol):
        hovered_symbol = symbol
        if(gdp_data.get(hovered_symbol) is not None):
            line.y = gdp_data[hovered_symbol].values
            fig_tooltip.title = content.get('ref_data', {}).get('Name', '')
               
# Custom msg sent when a cell is hovered on
market_map.on_hover(hover_handler)
market_map